In [1]:
import random
import json
import pickle
import numpy as np
import pandas as pd
import tensorflow
import keras
import nltk
#nltk.download()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, Activation, GlobalMaxPooling1D, Dropout, Bidirectional, LSTM
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
intents = json.loads(open('intents.json').read())

In [5]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
words = [word for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

### **Calculate X**

In [7]:
question = []
for doc in intents['intents']:
  question.append(doc['patterns'])

In [8]:
question_list = []
arr = [[question_list.append(x) for x in q] for q in question]

In [9]:
question_list

['Hi there',
 'How are you',
 'How are you?',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day',
 'Bye',
 'See you later',
 'Goodbye',
 'Nice chatting to you, bye',
 'Till next time',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Awesome, thanks',
 'Thanks for helping me',
 'what is your name',
 'what should I call you',
 'whats your name?',
 'who are you?',
 'how old',
 'old',
 'how old is florian',
 'what is your age',
 'old are you',
 'age?',
 'How you could help me?',
 'What you can do?',
 'What help you provide?',
 'How you can be helpful?',
 'What support is offered',
 'Can a student who has received a temporary certificate of completion be able to return to education?',
 'Can a graduate student who has a temporary certificate return to study?',
 'Can a graduate associate who has a temporary certificate return to study?',
 "What is the email address of the university's educational administration?",
 "What is the university's educational management email?",
 'Educational

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(question_list)

In [11]:
with open('tokenizer.h5', 'wb') as f:
    pickle.dump(tokenizer, f)

In [12]:
vocab_size = len(tokenizer.word_index)+1

In [13]:
vocab_size

570

In [14]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [15]:
data_to_sequences = tokenizer.texts_to_sequences(question_list)
maxlen = max_length(data_to_sequences)

In [16]:
maxlen

40

In [17]:
pad_data_to_sequences = pad_sequences(data_to_sequences, maxlen=maxlen)

In [18]:
pad_data_to_sequences

array([[  0,   0,   0, ...,   0, 424, 313],
       [  0,   0,   0, ...,  10,  13,  40],
       [  0,   0,   0, ...,  10,  13,  40],
       ...,
       [  0,   0,   0, ...,   1, 312,  64],
       [  0,   0,   0, ...,  82,  91, 421],
       [  0,   0,   0, ...,   1, 125, 423]], dtype=int32)

In [19]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

In [20]:
random.shuffle(training)
training = np.array(training)

<ipython-input-20-cc0bc1dae1db>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


### **Calculate Y**

In [21]:
tag = []
for doc in documents:
  tag.append(doc[1])

In [22]:
Y = pd.get_dummies(tag)

In [23]:
num_unique_tags = len(Y.nunique())

In [24]:
num_unique_tags

133

### **X and Y**

In [25]:
train_x = pad_data_to_sequences
train_y = np.array(Y)

### **Word Embedding Glove**

In [28]:
!wget https://nlp.stanford.edu/data/glove.840B.300d.zip

--2023-07-30 17:25:16--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2023-07-30 17:25:17--  https://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  5.02MB/s    in 6m 50s  

2023-07-30 17:32:07 (5.07 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [29]:
!unzip glove.840B.300d.zip

Archive:  glove.840B.300d.zip
  inflating: glove.840B.300d.txt     


**Load GloVe Word Embeddings in Memory**

In [30]:
%%time

import numpy as np

glove_embeddings = {}
with open("glove.840B.300d.txt") as f:
    for line in f:
        try:
            line = line.split()
            glove_embeddings[line[0]] = np.array(line[1:], dtype=np.float32)
        except:
            continue

CPU times: user 2min 49s, sys: 8.31 s, total: 2min 57s
Wall time: 3min 8s


**Create Matrix Of GloVe Embeddings for Vocabulary Tokens**

In [32]:
%%time

embedding_dim = 300

word_embeddings = np.zeros((len(tokenizer.index_word)+1, embedding_dim))

for idx, word in tokenizer.index_word.items():
    word_embeddings[idx] = glove_embeddings.get(word, np.zeros(embedding_dim))

CPU times: user 6.46 ms, sys: 996 µs, total: 7.46 ms
Wall time: 20.7 ms


### **Network**

In [34]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.index_word)+1, output_dim=embedding_dim,
                              input_length=maxlen, trainable=False, weights=[word_embeddings]),)
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_unique_tags, activation='softmax'))

In [35]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 300)           171000    
                                                                 
 bidirectional (Bidirectiona  (None, 40, 400)          801600    
 l)                                                              
                                                                 
 global_max_pooling1d (Globa  (None, 400)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 400)               0         
                                                                 
 dense (Dense)               (None, 200)               80200     
                                                                 
 dropout_1 (Dropout)         (None, 200)              

In [37]:
hist = model.fit(train_x, train_y, epochs=200, batch_size=128, verbose=1)

Epoch 1/200
4/4 [==============================] - 8s 423ms/step - loss: 4.9123 - accuracy: 0.0049
Epoch 2/200
4/4 [==============================] - 2s 408ms/step - loss: 4.8383 - accuracy: 0.0246
Epoch 3/200
4/4 [==============================] - 2s 635ms/step - loss: 4.7997 - accuracy: 0.0222
Epoch 4/200
4/4 [==============================] - 2s 481ms/step - loss: 4.7508 - accuracy: 0.0394
Epoch 5/200
4/4 [==============================] - 2s 400ms/step - loss: 4.6809 - accuracy: 0.0714
Epoch 6/200
4/4 [==============================] - 2s 422ms/step - loss: 4.6217 - accuracy: 0.0887
Epoch 7/200
4/4 [==============================] - 2s 412ms/step - loss: 4.5479 - accuracy: 0.1207
Epoch 8/200
4/4 [==============================] - 2s 402ms/step - loss: 4.4463 - accuracy: 0.1158
Epoch 9/200
4/4 [==============================] - 2s 404ms/step - loss: 4.2783 - accuracy: 0.1872
Epoch 10/200
4/4 [==============================] - 3s 740ms/step - loss: 4.1453 - accuracy: 0.1773
Epoch 11/

In [38]:
model.save('model.h5', hist)

## **Chatbot**

In [39]:
intents = json.loads(open('intents.json').read())

In [40]:
# load model
model = load_model('model.h5')

In [44]:
# load tokenizer
with open('tokenizer.h5', 'rb') as f:
    tokenizer = pickle.load(f)

In [45]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    return sentence_words

In [46]:
MAX_LENGTH = 40
def prediction(text):
  text_to_sequences = tokenizer.texts_to_sequences([text])
  pad_text_to_sequences = pad_sequences(text_to_sequences, maxlen=MAX_LENGTH)
  tag = model.predict(pad_text_to_sequences)[0]
  tag_index = np.where(tag == max(tag))[0][0]
  result = Y.columns[tag_index]
  return result

In [47]:
prediction('Can a student who has received a temporary certificate of completion be able to return to education')

1/1 [==============================] - 1s 907ms/step


'Can a student who has received a temporary certificate of completion be able to return to education?'

In [48]:
def get_response(text):
  tag = prediction(text)
  for data in intents['intents']:
    if data['tag'] == tag:
      result = random.choice(data['response'])
      break
  return result

In [49]:
get_response('Can a student who has received a temporary certificate of completion be able to return to education')

1/1 [==============================] - 0s 34ms/step


'If he receives a temporary certificate, he cannot return to study.'

In [ ]:
while True:
	message = input("Enter a message:")
	res = get_response(message)
	print(res)

Enter a message:Can a student who has received a temporary certificate of completion be able to return to education?
1/1 [==============================] - 0s 36ms/step
If he receives a temporary certificate, he cannot return to study.
Enter a message:What is in the temporary certificate of completion of education?
1/1 [==============================] - 0s 36ms/step
Due to the free education in the above-mentioned study level, daily students have the obligation to serve for free education based on the admission of the entrance exam area, and all obligations must be settled when receiving the educational encyclopedia. (Complete description of settlement of obligations in the receiving section of the encyclopedia)
